In [ ]:
#Import Dependencies 
import pandas as pd
from sqlalchemy import create_engine
import psycopg2


In [ ]:
#Import CSV
CitiBike = "../ETL/CitiBike.csv"
CitiBike = pd.read_csv("CitiBike.csv")
CitiBike_df = CitiBike.assign(DATEINT = CitiBike['Date'].str.replace("/","").astype(int))
CitiBike_df.head()

In [ ]:
#Grouped csv data by date in order to preserve hierarchy amongst the scattered weather data. 
Weather = pd.read_csv('1784274.csv')
Weather = Weather.assign(DATEINT = Weather['DATE'].str.replace("/","").astype(int))
Weather = Weather.groupby(['DATE']).mean()
Weather = Weather.reset_index()
FilterWeather = Weather[['DATEINT','PRCP','SNOW','SNWD','TMAX','TMIN']]
FilterWeather = FilterWeather.rename(columns= {'DATEINT':'dateint','PRCP':'percip_in_inches','SNOW':'snow_in_inches','SNWD':'snow_depth_in_inches','TMAX':'max_temp','TMIN':'min_temp'})
FilterWeather.head()


In [ ]:
#Cleaned bike data for easier data retrieval. Names were too long and were shortened for simple queries.
CitiBike = pd.read_csv("CitiBike.csv")
CitiBike_df = CitiBike.assign(DATEINT = CitiBike['Date'].str.replace("/","").astype(int))
# CitiBike_df.head()

CitiBike_Cleaned1 = CitiBike_df.rename(columns={'Trips over the past 24-hours (midnight to 11:59pm)': 'daily_trips',
                                               'Cumulative trips (since launch):': 'cumulative_trips',
                                               'Miles traveled today (midnight to 11:59 pm)': 'daily_miles',
                                               'Miles traveled to date:': 'cumulative_miles',
                                               'Total Annual Members': 'cumulative_members'
                                               })
CitiBike_Cleaned2 = CitiBike_Cleaned1.drop(['Date',
                                            'Annual Member Sign-Ups (midnight to 11:59 pm)',
                                            '24-Hour Passes Purchased (midnight to 11:59 pm)',
                                            '7-Day Passes Purchased (midnight to 11:59 pm)',
                                            '3-Day Passes Purchased (midnight to 11:59 pm)'], axis = 1)
CitiBike_df = CitiBike_Cleaned1
CitiBike_df = CitiBike_df[['DATEINT','daily_trips','cumulative_trips','daily_miles','cumulative_miles','cumulative_members']]
CitiBike_df = CitiBike_df.rename(columns= {'DATEINT':'dateint'})

In [ ]:
CitiBike_df.head()

In [ ]:
connection_string = "postgres:password@localhost:5432/CitiBike_db"
engine = create_engine(f'postgresql://{connection_string}')

In [ ]:
# Confirm tables
engine.table_names()

In [ ]:
CitiBike_df.to_sql(name='citibike', con=engine, if_exists='append', index=True)

In [ ]:
FilterWeather.to_sql(name='weather', con=engine, if_exists='append', index=True)